In [2]:
# adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_train_30.h5ad"
# adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_test_30.h5ad"
# adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_ood_30.h5ad"

In [142]:
import anndata as ad
import numpy as np
import biolord

from parameters_sciplex3 import module_params, trainer_params
from utils_perturbation_sciplex3 import compute_prediction

In [4]:
adata = ad.read_h5ad("../data/sciplex/sciplex3_biolord.h5ad")

In [6]:
biolord.Biolord.setup_anndata(
    adata,
    ordered_attributes_keys=["rdkit2d_dose"],
    categorical_attributes_keys=["cell_type"],
    retrieval_attribute_key=None,
)

INFO     Generating sequential column names                                                                        


In [9]:
model = biolord.Biolord(
    adata=adata,
    n_latent=256,
    model_name="sciplex3",
    module_params=module_params,
    train_classifiers=False,
    split_key="split_ood",
)

Seed set to 42


In [16]:
model.train(
    max_epochs=1,#200
    batch_size=512,
    plan_kwargs=trainer_params,
    early_stopping=True,
    early_stopping_patience=20,
    check_val_every_n_epoch=10,
    num_workers=10,
    enable_checkpointing=False
)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/arturszalata/miniconda3/envs/ot_pert_reproducibility/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/arturszalata/miniconda3/envs/ot_pert_reproducibility/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
/Users/arturszalata/miniconda3/envs/ot_pert_reproducibility/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Epoch 1/1: 100%|██████████| 1/1 [08:02<00:00, 482.98s/it, v_num=1, val_generative_mean_accuracy=0.399, val_generative_var_accuracy=-4.76, val_biolord_metric=-2.18, val_reconstruction_loss=424, val_unknown_attribute_penalty_loss=204]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 1/1: 100%|██████████| 1/1 [08:03<00:00, 483.01s/it, v_num=1, val_generative_mean_accuracy=0.399, val_generative_var_accuracy=-4.76, val_biolord_metric=-2.18, val_reconstruction_loss=424, val_unknown_attribute_penalty_loss=204]


In [95]:
idx_test_control = np.where(
    (adata.obs["split_ood"] == "test") & (adata.obs["control"] == 1)
)[0]

adata_test_control = adata[idx_test_control].copy()

idx_ood = np.where((adata.obs["split_ood"] == "ood"))[0]

adata_ood = adata[idx_ood].copy()

In [143]:
# narrow to a single drug dose cell type for testing
adata_ood = adata_ood[(adata_ood.obs.cov_drug_dose_name == "A549_Quisinostat_0.001")]

In [138]:
dataset_control = model.get_dataset(adata_test_control)
dataset_ood = model.get_dataset(adata_ood)

INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


In [140]:
biolord_prediction = compute_prediction(
    model=model,
    adata=adata_ood,
    dataset=dataset_ood,
    dataset_control=dataset_control)

0it [00:00, ?it/s]/var/folders/98/9ygjwkpd12d1vxydfprn1st00000gp/T/ipykernel_28888/253641653.py:9: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(arr).nonzero()` if the old behavior was intended.
  return np.where(x)[0]
1it [00:00,  4.73it/s]
